# Hausaufgabe 3 (11P)


### Hinweise zur Abgabe

Abgabetermin ist **Mittwoch, 20. Juli 2022, 23:55 Uhr**. Die Abgabe erfolgt über die Upload-Option im ISIS-Kurs. Verspätete Abgaben bzw. Abgaben, die nicht über das ISIS-Portal erfolgen, werden für die Bewertung nicht berücksichtigt. Pro Gruppe wird nur eine Abgabe hochgeladen.

Die Hausaufgabe soll in Form eines Jupyter Notebooks und der Unity-Dateien (s. Hinweis unten) als ZIP-Archiv abgegeben werden. Dazu können Sie dieses Notebook als Vorlage benutzen. Das Notebook soll dabei sowohl alle Programmieraufgaben als auch alle schriftlich zu beantwortenden Aufgaben umfassen. Programmieraufgaben sollen als Code-Zelle implementiert werden. Schriftliche Aufgaben können mit Markdown-Zellen beantwortet werden. Grafiken sollen jeweils durch die Code-Zellen erzeugt werden. Achten Sie auf eine korrekte und vollständige Beschriftung der Grafiken (Achsen, Legenden). Im Notebook sind außerdem Namen und Matrikelnummern aller Gruppenmitglieder anzugeben.

Beachten Sie, dass das Notebook ausführbar und nachvollziehbar kommentiert ist. Ihre Abgabe muss alle Dateien umfassen, die zur fehlerfreien Ausführung des Notebooks notwendig sind. Alle abzugebenden Dateien müssen als ZIP-Datei komprimiert und mit der Namenskonvention Elektroakustik_SoSe22_HA2_GruppeXX versehen werden (ersetzen Sie XX entsprechend durch Ihre Gruppe, z.B. GruppeA).

Zur Lösung der Hausaufgabe können u.a. folgende Module verwendet werden:

In [2]:
import numpy as np
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='svg' 
import pyfar as pf
import sounddevice as sd

### Aufgabe 1 (3P)

Wie groß sind Wandimpedanz und Absorptionsgrad einer Schicht aus Schaumstoff mit einem Strömungswiderstand von Ξ = 2⋅10^4   Ns∕m^4  und einer Porösität von 𝜎 = 0,97 mit einer Schichtdicke von 10 cm vor einer schallharten Wand bei den Frequenzen 300 Hz und 1000 Hz?

$\alpha=\frac{4 \operatorname{Re}\left\{\frac{z}{\rho_{0} c}\right\}}{\left[\operatorname{Re}\left\{\frac{z}{\rho_{0} c}\right\}+1\right]^{2}+\left[\operatorname{Im} \left\{\frac{z}{\rho_{0} c}\right\}\right]^{2}}$

mit 

$k^{\prime}=\frac{\omega}{c} \sqrt{1-\frac{j \sigma \Xi}{\rho_{0}c}}$
,

$z_{0 e}=\frac{\rho_{0} c}{\sigma} \sqrt{1-\frac{j \sigma \Xi}{\rho_{0 \omega}}}$

und

$z=-j z_{0 e} \cot \left(k^{\prime} d\right)$

ist

$\alpha = \frac{4 \operatorname{Re}\left\{\frac{-j\left(\frac{\rho_{0} c}{\sigma} 1-\frac{j \sigma \Xi}{\rho_{0} \omega}\right) \cot \left(d \cdot\left[\frac{\omega}{c} (1-\frac{j \sigma \Xi}{\rho_{0} c})\right]\right)}{\rho_{0} c}\right\}} 
{\bigg[ \operatorname{Re}\left\{\frac{-j\left(\frac{\rho_{0} c}{\sigma} 1-\frac{j \sigma \Xi}{\rho_{0} \omega}\right) \cot \left(d \cdot\left[\frac{\omega}{c} (1-\frac{j \sigma \Xi}{\rho_{0} c})\right]\right)}{\rho_{0} c}\right\} + 1\bigg]^2 + 
\bigg[ \operatorname{Im}\left\{\frac{-j\left(\frac{\rho_{0} c}{\sigma} 1-\frac{j \sigma \Xi}{\rho_{0} \omega}\right) \cot \left(d \cdot\left[\frac{\omega}{c} (1-\frac{j \sigma \Xi}{\rho_{0} c})\right]\right)}{\rho_{0} c}\right\}\bigg]^2}$


In [18]:
fs = np.array([300, 1000])
ws = 2*np.pi*fs
c = 343.2
rho_0 = 1.2041
sigma = 0.97
Xi = 2e+4
d = 0.1

def calc_k_complex(omega):
    return((omega/c) * np.sqrt(1 - (1j*sigma*Xi/(rho_0*c))))

def calc_z0_e(omega):
    return(rho_0*c/sigma*np.sqrt(1 - (1j*sigma*Xi/(rho_0*omega))))

def calc_z(k_complex, z0_e):
    return(-1j*z0_e*(1/np.tan(k_complex*d)))

alphas = []
zs = []
for w in ws:
    k_complex = calc_k_complex(w)
    z0_e = calc_z0_e(w)
    z = calc_z(k_complex, z0_e)
    alpha = 4*np.real(z/(rho_0*c)) / ((np.real(z/(rho_0*c)) + 1)**2 + (np.imag(z/(rho_0*c)))**2)
    alphas.append(alpha)
    zs.append(z)

print(alphas)
print(zs)

[0.6171183096379277, 0.8368709814048154]
[(946.4051137827133-828.4616934904898j), (583.5444042226595-398.7783898925541j)]


| Frequenz in Hz | Wandimpedanz $z$ in $\mathrm{\frac{Ns}{m^3}}$ | Absorptionsgrad $\alpha$ |
| --- | --- | --- |
| 300| 946,41-828,46j | 0.617 |
| 1000 | 583,54-398,78j | 0,837 |

### Aufgabe 2 (3P)
Ein Resonanzabsorber, realisiert durch einen Plattenabsorber vor einem geschlossenen Lufvorlumen, soll auf die Resonanzfrequenz von 300 Hz (500 Hz) eingestellt werden. Für das Luftvolumen ist eine Schichtdicke von 5 cm vorgesehen. Welche Massenbeläge sind erforderlich? Wie wirkt sich eine Erhöhung des Massenbelags 𝑚′ und eine Verringerung der Schichtdicke 𝑑, so dass 𝑚′ 𝑑=const., auf die Lage der Resonanzfrequenz und auf die Güte der Resonanz, d.h. die Breite des Resonanzgipfels, aus?

| Resonanz-Frequenz $f_0$ in Hz | erforderlicher Massenbelag $m'$ in $\frac{kg}{m^2}$ |
| --- | --- |
| 300 | 80 |
| 500 | 28,7 |

Für die Resonanzfrequenz gilt $f_0 = \sqrt{\frac{\rho_0 c}{m' d}}$ d.h. solange $m'd = $ const, bleibt die Lage der Resonanzfrequenz unverändert.

### Aufgabe 3 (2P)
In welchem Frequenzbereich liegt in einem Kundtschen Rohr mit einem quadratischen Querschnitt von 10 cm Seitenlänge ein vollständig ebenes Schallfeld vor, bei dem sich der Schalldruck nur in Längsrichtung des Rohrs verändert?

Die Cutoff Frequenz im rechteckigen Rohr ist:
$ f_{c 1}=\frac{c}{2 h} $, wobei h die größte Seitenlänge des Querschnitts ist.
Eingesetzt ergibt sich $ f_{c 1}=\frac{343.2}{2 * 0.2 m}=858 Hz $

### Aufgabe 4: Unity & Oculus Spatializer (3P)

Installieren Sie Unity in der Version 2021.3.5f1 und den Oculus Spatializer. Folgen Sie hierfür der Anleitung auf dieser Seite: https://developer.oculus.com/documentation/unity/audio-osp-unity-req-setup/?locale=de_DE.

1. In Unity navigieren Sie zum Ordner: `/Assets/Oculus/Spatializer/scenes` und legen Sie hier den Unterordner `Hausaufgabe3` an. Kopieren Sie die Sample Scene "RedBallGreenBall" (https://developer.oculus.com/documentation/unity/audio-osp-unity-scene/) in den zuvor angelegten Unterordner und nennen Sie die Kopie "Hausaufgabe3". Alle folgenden Schritte sollen innerhalb des Ordners `Hausaufgabe3` und innerhalb der eben kopierten, gleichnamigen Szene erfolgen.


2. Importieren Sie die Datein `GewandHausPublikum.fbx` sowie `GewandHausRest.fbx` und fügen Sie die Objekte gleichzeitig der Szene hinzu.


3. Importieren Sie den Ordner `Quartett`. Löschen sie die rote Quelle. Verteilen Sie 4 Kopien der grünen Quellen auf der Bühne im Gewandhaus, nennen Sie diese Violine1, Violine2, Bratsche und Cello und ordnen Sie diesen jeweils die entsprechenden Wav-Dateien des Streichquartetts zu.


4. Erstellen Sie ein Objekt "Camera" und fügen Sie diesem das Skript "FlyCamera" hinzu.


5. Wählen Sie die Objekte `GewandHausPublikum` und `GewandHausRest` in der Szene aus und fügen Sie diesem die Komponenten "ONSP Propagation Geometry (Script)" und "ONSP Propagation Material (Script)" hinzu.


6. Starten Sie die Szene (Play), navigieren Sie durch das Gewandhaus und hören Sie sich die Auralisation an.


7. Verlassen Sie den Play-Modus und wählen Sie die jeweiligen Komponenten "ONSP Propagation Material (Script)" das Sie vorher dem GewandHaus-Objekt hinzugefügt haben aus und Ordnen Sie die folgenden Absorptionskoeffizienten von 125Hz bis 4 kHz grob zu (Scattering und Transmission kann so belassen werden):


![Absorptionskoeffizienten](absorption.png)

8. Wählen Sie jeweils im "ONSP Propagation Geometry (Script)" desselben Objekts "File Enabled" aus und klicken Sie auf "Bake Mesh to File".


9. Starten Sie nun die Szene erneut und hören Sie sich die Auralisation an.

### Hinweis zur Abgabe

Fügen Sie den in Unity erstellten Ordner `Hausaufgabe 3` Ihrem Upload hinzu. Löschen Sie vorher bitte den Unterordner `Quartett`, sodass die maximale Dateigröße nicht überschritten wird.